In [2]:
from creds import HOST_NAME, DB_NAME, USERNAME , PASSWORD 
import pandas as pd
import duckdb as duck
import psycopg
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def exec_query_os(query):
    conn = psycopg.connect(
        dbname=DB_NAME, user=USERNAME, password=PASSWORD, host=HOST_NAME, port="5432"
    )
    df = pd.read_sql(query, conn)

    conn.close()
    return df

def df_to_parquet(df, table_name):
    df.to_parquet(f"./data/{table_name}.parquet", compression='gzip')


## Reading NBA Data from db

In [3]:
game_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_GAME_BOX_HIST"
                where season = 2024;
            """)

game_pbp_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PBP_HIST"
                where season = 2024;
            """)

player_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PLAYER_BOX_HIST"
                where season = 2024;
            """)


In [4]:
print(game_box_df.shape)
print(game_pbp_df.shape)
print(player_box_df.shape)


(2628, 57)
(611682, 64)
(34867, 57)


## Cleaning and Normalizing Data

### Team Game Stats

In [5]:
team_schedules = game_box_df.copy()

# Creating a home away 'indicator' - mainly for tooltip
team_schedules['game_title'] = np.where(team_schedules['team_home_away'] == 'home',
                                      team_schedules['opponent_team_abbreviation'] + ' @ ' + team_schedules['team_abbreviation'],
                                      team_schedules['team_abbreviation'] + ' @ ' + team_schedules['opponent_team_abbreviation']
                                    )

# Creating win loss 'indicator' - mainly for tooltip
team_schedules['outcome_title'] = np.where(team_schedules['team_winner'] == True,
                                      'W ' + team_schedules['team_score'].astype(str) + '-' + team_schedules['opponent_team_score'].astype(str),
                                      'L ' + team_schedules['opponent_team_score'].astype(str) + '-' + team_schedules['team_score'].astype(str)
                                    )

print(team_schedules.shape)
team_schedules.head()

(2628, 59)


,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,...,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score,largest_lead,game_title,outcome_title
0,401656363,2024,3,2024-06-17,2024-06-18 00:30:00+00:00,6,s:40~l:46~t:6,dallas-mavericks,Dallas,Mavericks,...,BOS,Boston Celtics,Celtics,008348,ffffff,https://a.espncdn.com/i/teamlogos/nba/500/bos.png,106,0,DAL @ BOS,L 106-88
1,401656363,2024,3,2024-06-17,2024-06-18 00:30:00+00:00,2,s:40~l:46~t:2,boston-celtics,Boston,Celtics,...,DAL,Dallas Mavericks,Mavericks,0064b1,bbc4ca,https://a.espncdn.com/i/teamlogos/nba/500/dal.png,88,26,DAL @ BOS,W 106-88
2,401656362,2024,3,2024-06-14,2024-06-15 00:30:00+00:00,2,s:40~l:46~t:2,boston-celtics,Boston,Celtics,...,DAL,Dallas Mavericks,Mavericks,0064b1,bbc4ca,https://a.espncdn.com/i/teamlogos/nba/500/dal.png,122,3,BOS @ DAL,L 122-84
3,401656362,2024,3,2024-06-14,2024-06-15 00:30:00+00:00,6,s:40~l:46~t:6,dallas-mavericks,Dallas,Mavericks,...,BOS,Boston Celtics,Celtics,008348,ffffff,https://a.espncdn.com/i/teamlogos/nba/500/bos.png,84,48,BOS @ DAL,W 122-84
4,401656361,2024,3,2024-06-12,2024-06-13 00:30:00+00:00,2,s:40~l:46~t:2,boston-celtics,Boston,Celtics,...,DAL,Dallas Mavericks,Mavericks,0064b1,bbc4ca,https://a.espncdn.com/i/teamlogos/nba/500/dal.png,99,21,BOS @ DAL,W 106-99


In [ ]:
# normalizing team game dataframe
# 4 final dataframes

# splitting team info into separate table
team_lkp = team_schedules.copy()
team_lkp = team_lkp[['team_id', 'team_slug', 'team_location', 'team_name',
       'team_abbreviation', 'team_display_name', 'team_short_display_name',
       'team_color', 'team_alternate_color', 'team_logo']].drop_duplicates().reset_index(drop=True)

# splitting game outcome into separate table
game_overview = team_schedules.copy()
game_overview = game_overview[['game_id', 'season', 'season_type', 'game_date', 'game_date_time', 'game_title']].drop_duplicates().reset_index(drop=True)

# splitting home and away into separate table
home_stats = team_schedules.copy()
home_stats = home_stats[home_stats['team_home_away'] == 'home'][['game_id', 'team_id',
                                                               'team_score', 'team_winner', 'assists', 'blocks', 
                                                               'defensive_rebounds', 'fast_break_points', 'field_goal_pct', 
                                                               'field_goals_made', 'field_goals_attempted', 'flagrant_fouls', 
                                                               'fouls', 'free_throw_pct','free_throws_made', 'free_throws_attempted', 
                                                               'offensive_rebounds','points_in_paint', 'steals', 'team_turnovers', 
                                                               'technical_fouls','three_point_field_goal_pct', 'three_point_field_goals_made',
                                                               'three_point_field_goals_attempted', 'total_rebounds','total_technical_fouls', 
                                                               'total_turnovers', 'turnover_points', 'turnovers']].reset_index(drop=True)

away_stats = team_schedules.copy()
away_stats = away_stats[away_stats['team_home_away'] == 'away'][['game_id', 'team_id',
                                                               'team_score', 'team_winner', 'assists', 'blocks', 
                                                               'defensive_rebounds', 'fast_break_points', 'field_goal_pct', 
                                                               'field_goals_made', 'field_goals_attempted', 'flagrant_fouls', 
                                                               'fouls', 'free_throw_pct','free_throws_made', 'free_throws_attempted', 
                                                               'offensive_rebounds','points_in_paint', 'steals', 'team_turnovers', 
                                                               'technical_fouls','three_point_field_goal_pct', 'three_point_field_goals_made',
                                                               'three_point_field_goals_attempted', 'total_rebounds','total_technical_fouls', 
                                                               'total_turnovers', 'turnover_points', 'turnovers']].reset_index(drop=True)
# adding team id's to game overview table
final_game_overview = pd.merge(
       pd.merge(
              game_overview, home_stats[['game_id', 'team_id']], on='game_id', how='left'
       ).rename(columns={'team_id':'home_team_id'}),
       away_stats[['game_id', 'team_id']], on='game_id', how='left'
       ).rename(columns={'team_id':'away_team_id'}).reset_index(drop=True)

print(f"Team Lookup Table Shape: {team_lkp.shape}")
print(f"Game Overview Table Shape: {final_game_overview.shape}")
print(f"Home Team Stats Table Shape: {home_stats.shape}")
print(f"Away Team Stats Table Shape: {away_stats.shape}")

Team Lookup Table Shape: (32, 11)
Game Overview Table Shape: (1314, 8)
Home Team Stats Table Shape: (1314, 29)
Away Team Stats Table Shape: (1314, 29)


### Player Game Stats

In [7]:
# Creating a player info table
player_lkp = player_box_df.copy()
player_lkp = player_lkp[['athlete_id', 'athlete_display_name', 'athlete_headshot_href',
                               'athlete_position_name', 'athlete_position_abbreviation']].drop_duplicates().reset_index(drop=True)

# Creating a link to team
player_team_lkp = player_box_df.copy()
player_team_lkp = player_team_lkp[['athlete_id','team_id', 'game_date']].drop_duplicates()
player_team_lkp = player_team_lkp.groupby(['athlete_id','team_id'], as_index=False).agg({
                'game_date': ['min', 'max']
})
player_team_lkp.columns = ['athlete_id', 'team_id', 'first_game_date', 'last_game_date']
player_team_lkp = player_team_lkp.reset_index(drop=True)

# game player stat
player_game_stat= player_box_df.copy()
player_game_stat = player_game_stat[['game_id','athlete_id', 'minutes', 'field_goals_made', 'field_goals_attempted', 
                                     'three_point_field_goals_made', 'three_point_field_goals_attempted', 'free_throws_made',
                                     'free_throws_attempted', 'offensive_rebounds', 'defensive_rebounds', 'rebounds', 'assists',
                                     'steals', 'blocks', 'turnovers', 'fouls', 'plus_minus', 'points', 'starter', 'ejected',
                                     'did_not_play', 'active', 'reason']].drop_duplicates().reset_index(drop=True)

print(f"Player Lookup Table Shape: {player_lkp.shape}")
print(f"player Team Info Table Shape: {player_team_lkp.shape}")
print(f"Game Player Stats Table Shape: {player_game_stat.shape}")

Player Lookup Table Shape: (589, 5)
player Team Info Table Shape: (704, 4)
Game Player Stats Table Shape: (34867, 24)


### Play by Play Player Stats

In [31]:
# Normalizing player pbp data

# grouping by event types
type_1 = [7, 8, 9, 10, 11, 12, 13, 22, 24, 25, 28, 29, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 44, 45, 46, 47, 48, 257]
type_2 = [16, 213, 214, 215, 216, 277, 278, 279, 280, 402, 412, 517, 584, 615]
type_3 = [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 86, 87, 88, 90, 206]
type_4 = [91, 92, 93, 94, 95, 96, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
          126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
          148, 149, 150, 151, 152, 153]
type_5 = [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 157, 165, 166]
type_6 = [155, 156]

game_pbp_frmt = game_pbp_df.copy()
game_pbp_frmt['period_id'] = game_pbp_frmt['game_id'].astype(str) + game_pbp_frmt['period_number'].astype(str)

type_group_conditions = [
            game_pbp_frmt['type_id'] == 0,
            game_pbp_frmt['type_id'].isin(type_1),
            game_pbp_frmt['type_id'].isin(type_2),
            game_pbp_frmt['type_id'].isin(type_3),
            game_pbp_frmt['type_id'].isin(type_4),
            game_pbp_frmt['type_id'].isin(type_5),
            game_pbp_frmt['type_id'].isin(type_6)
]
game_pbp_frmt['type_group_id'] = np.select(type_group_conditions, [0, 1, 2, 3, 4, 5, 6], default=0)

type_group_desc_conditions = [
            game_pbp_frmt['type_group_id'] == 0,
            game_pbp_frmt['type_group_id'] == 1,
            game_pbp_frmt['type_group_id'] == 2,
            game_pbp_frmt['type_group_id'] == 3,
            game_pbp_frmt['type_group_id'] == 4,
            game_pbp_frmt['type_group_id'] == 5,
            game_pbp_frmt['type_group_id'] == 6
]
type_group_descs = ['None', 'foul', 'pause', 'turnover', 'shot', 'free-throw', 'rebound']
game_pbp_frmt['type_group_text'] = np.select(type_group_desc_conditions, type_group_descs, default='None')

# game period lookup
game_period_lkp = game_pbp_frmt.copy()
game_period_lkp = game_period_lkp[['game_id', 'period_id', 'period_number']].drop_duplicates().sort_values(
                    by=['game_id', 'period_id'], ascending=True
                ).reset_index(drop=True)

# period time info lookup
period_time = game_pbp_frmt.copy()
period_time = period_time[['period_id', 'id', 'sequence_number', 'wallclock',
                           'clock_display_value', 'start_quarter_seconds_remaining', 'start_half_seconds_remaining',
                           'start_game_seconds_remaining', 'game_play_number', 'end_quarter_seconds_remaining', 'end_half_seconds_remaining',
                           'end_game_seconds_remaining']].drop_duplicates().reset_index(drop=True)

# game score lookup
game_score_lkp = game_pbp_frmt.copy()
game_score_lkp = game_score_lkp[['period_id', 'id', 'sequence_number',
                                 'home_score', 'away_score']].drop_duplicates().reset_index(drop=True)

# event type lookup
event_type_lkp = game_pbp_frmt.copy()
event_type_lkp = event_type_lkp[['type_group_id', 'type_group_text',
                                 'type_id', 'type_text']].drop_duplicates().reset_index(drop=True)

# play overview 
play_overview = game_pbp_frmt.copy()
play_overview = play_overview[['id', 'type_id', 'team_id', 'scoring_play', 'score_value', 'shooting_play',
                               'athlete_id_1', 'athlete_id_2', 'athlete_id_3', 'coordinate_x',
                               'coordinate_y']].drop_duplicates().reset_index(drop=True)

print(f"Game Period Lookup Table Shape: {game_period_lkp.shape}")
print(f"Period Time Info Table Shape: {period_time.shape}")
print(f"Game Score Lookup Table Shape: {game_score_lkp.shape}")
print(f"Event Type Lookup Table Shape: {event_type_lkp.shape}")
print(f"Player Play Stat Table Shape: {play_overview.shape}")

Game Period Lookup Table Shape: (5326, 3)
Period Time Info Table Shape: (611682, 12)
Game Score Lookup Table Shape: (611682, 5)
Event Type Lookup Table Shape: (138, 4)
Player Play Stat Table Shape: (611682, 11)


## Change Data Types and Export to parquet

In [60]:
# Adjusting data types according to ERD

GAME_PERIOD_LKP = game_period_lkp.copy()
GAME_PERIOD_LKP['period_id'] = GAME_PERIOD_LKP['period_id'].astype(int)

PERIOD_INFO_LKP = period_time.copy()
PERIOD_INFO_LKP['period_id'] = PERIOD_INFO_LKP['period_id'].astype(int)
PERIOD_INFO_LKP['id'] = PERIOD_INFO_LKP['id'].astype(int)
PERIOD_INFO_LKP['sequence_number'] = PERIOD_INFO_LKP['sequence_number'].astype(int)
PERIOD_INFO_LKP['wallclock'] = pd.to_datetime(PERIOD_INFO_LKP['wallclock'])
PERIOD_INFO_LKP['clock_display_value'] = PERIOD_INFO_LKP['clock_display_value'].astype(str)

GAME_PLAY_SCORE_LKP = game_score_lkp.copy()
GAME_PLAY_SCORE_LKP['period_id'] = GAME_PLAY_SCORE_LKP['period_id'].astype(int)
GAME_PLAY_SCORE_LKP['id'] = GAME_PLAY_SCORE_LKP['id'].astype(int)
GAME_PLAY_SCORE_LKP['sequence_number'] = GAME_PLAY_SCORE_LKP['sequence_number'].astype(int)

PLAY_TYPE_LKP = event_type_lkp.copy()
PLAY_TYPE_LKP['type_group_text'] = PLAY_TYPE_LKP['type_group_text'].astype(str)
PLAY_TYPE_LKP['type_text'] = PLAY_TYPE_LKP['type_text'].astype(str)

PLAY_INFO_OVERVIEW = play_overview.copy()
PLAY_INFO_OVERVIEW['id'] = PLAY_INFO_OVERVIEW['id'].astype(int)
PLAY_INFO_OVERVIEW['team_id'] = PLAY_INFO_OVERVIEW['team_id'].fillna(0)
PLAY_INFO_OVERVIEW['team_id'] = PLAY_INFO_OVERVIEW['team_id'].astype(int)

ATHLETE_INFO_LKP = player_lkp.copy()
ATHLETE_INFO_LKP['athlete_display_name'] = ATHLETE_INFO_LKP['athlete_display_name'].astype(str)
ATHLETE_INFO_LKP['athlete_position_name'] = ATHLETE_INFO_LKP['athlete_position_name'].astype(str)
ATHLETE_INFO_LKP['athlete_position_abbreviation'] = ATHLETE_INFO_LKP['athlete_position_abbreviation'].astype(str)

ATHLETE_ACTIVE_TEAM_LKP = player_team_lkp.copy()
ATHLETE_ACTIVE_TEAM_LKP['first_game_date'] = pd.to_datetime(ATHLETE_ACTIVE_TEAM_LKP['first_game_date'])
ATHLETE_ACTIVE_TEAM_LKP['first_game_date'] = ATHLETE_ACTIVE_TEAM_LKP['first_game_date'].dt.date
ATHLETE_ACTIVE_TEAM_LKP['last_game_date'] = pd.to_datetime(ATHLETE_ACTIVE_TEAM_LKP['last_game_date'])
ATHLETE_ACTIVE_TEAM_LKP['last_game_date'] = ATHLETE_ACTIVE_TEAM_LKP['last_game_date'].dt.date

PLAYER_GAME_STATS = player_game_stat.copy()
PLAYER_GAME_STATS['plus_minus'] = PLAYER_GAME_STATS['plus_minus'].astype(str)
PLAYER_GAME_STATS['reason'] = PLAYER_GAME_STATS['reason'].astype(str)

TEAM_INFO_LKP = team_lkp.copy()

GAME_INFO_OVERVIEW = final_game_overview.copy()
GAME_INFO_OVERVIEW['game_date'] =  pd.to_datetime(GAME_INFO_OVERVIEW['game_date'])
GAME_INFO_OVERVIEW['game_date'] =  GAME_INFO_OVERVIEW['game_date'].dt.date

GAME_INFO_OVERVIEW['game_date_time'] = pd.to_datetime(GAME_INFO_OVERVIEW['game_date'])

HOME_TEAM_GAME_STATS = home_stats.copy()
HOME_TEAM_GAME_STATS['fast_break_points'] = HOME_TEAM_GAME_STATS['fast_break_points'].astype(int)

AWAY_TEAM_GAME_STATS = away_stats.copy()
AWAY_TEAM_GAME_STATS['fast_break_points'] = AWAY_TEAM_GAME_STATS['fast_break_points'].astype(int)


In [65]:
# Sending tables to parquet files

GAME_PERIOD_LKP.to_parquet('./data/GAME_PERIOD_LKP.parquet', engine='pyarrow')
PERIOD_INFO_LKP.to_parquet('./data/PERIOD_INFO_LKP.parquet', engine='pyarrow')
GAME_PLAY_SCORE_LKP.to_parquet('./data/GAME_PLAY_SCORE_LKP.parquet', engine='pyarrow')
PLAY_TYPE_LKP.to_parquet('./data/PLAY_TYPE_LKP.parquet', engine='pyarrow')
PLAY_INFO_OVERVIEW.to_parquet('./data/PLAY_INFO_OVERVIEW.parquet', engine='pyarrow')
ATHLETE_INFO_LKP.to_parquet('./data/ATHLETE_INFO_LKP.parquet', engine='pyarrow')
ATHLETE_ACTIVE_TEAM_LKP.to_parquet('./data/ATHLETE_ACTIVE_TEAM_LKP.parquet', engine='pyarrow')
PLAYER_GAME_STATS.to_parquet('./data/PLAYER_GAME_STATS.parquet', engine='pyarrow')
TEAM_INFO_LKP.to_parquet('./data/TEAM_INFO_LKP.parquet', engine='pyarrow')
GAME_INFO_OVERVIEW.to_parquet('./data/GAME_INFO_OVERVIEW.parquet', engine='pyarrow')
HOME_TEAM_GAME_STATS.to_parquet('./data/HOME_TEAM_GAME_STATS.parquet', engine='pyarrow')
AWAY_TEAM_GAME_STATS.to_parquet('./data/AWAY_TEAM_GAME_STATS.parquet', engine='pyarrow')


## Testing Analysis and Visuals

In [58]:
# duck.read_parquet('')
query = """
SELECT 
    athlete_display_name,
    team_name,
    min(game_date) as first_game_date,
    max(game_date) as last_game_date
FROM player_game_info 
--WHERE athlete_id = '3012'
GROUP BY athlete_display_name, team_name
"""

# Execute query with DuckDB
result = duck.query(query).df()
result.head()

,athlete_display_name,team_name,first_game_date,last_game_date
0,Xavier Tillman,Celtics,2024-02-22,2024-06-17
1,Oshae Brissett,Celtics,2023-10-25,2024-06-17
2,Svi Mykhailiuk,Celtics,2023-10-25,2024-06-17
3,Mike Conley,Timberwolves,2023-10-25,2024-05-30
4,Naz Reid,Timberwolves,2023-10-25,2024-05-30
